In [1]:
import torch
from torch.utils.data.dataset import Dataset
from torchvision.datasets.mnist import MNIST
from torchvision.transforms import ToTensor


In [2]:

USE_CUDA = torch.cuda.is_available() # GPU를 사용가능하면 True, 아니라면 False를 리턴
device = torch.device("cuda" if USE_CUDA else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용
print("다음 기기로 학습합니다:", device)

다음 기기로 학습합니다: cpu


In [3]:


mnist_train = MNIST(root='MNIST_data/',train=True,transform=ToTensor(),download = True)
mnist_test = MNIST(root='MNIST_data/',train=False,transform=ToTensor(),download = True)

In [4]:
#model MNist:
#   input_flatten = mnist_train.train_data.float().reshape(mnist_train.train_data.shape[0],-1)
#   out = input_flatten[1] @ ?ToNodes(10)
#   out_predict = torch.argmax(out)
#   input_flatten[0] ?= out_predict

In [7]:

from pyplus.pytorch.simple import TorchPlus

tp = TorchPlus()
tp.meta_optimizer = torch.optim.SGD
tp.meta_optimizer_epoch = 300
tp.meta_optimizer_learning_rate = 0.01
tp.meta_error_measurement = torch.nn.MSELoss
def assign_values(self):
    self.all_tensors['input_flatten'] = mnist_train.train_data.float().reshape(mnist_train.train_data.shape[0],-1)
    self.all_tensors['___p_0']  = torch.rand(self.all_tensors['input_flatten'][1].shape[-1],10,require_grads=True)
tp.assign_process_values=assign_values
def assign_process(self):
    input_flatten = self.all_tensors['input_flatten']
    out = input_flatten[1] @ self.all_tensors['___p_0']
    out_predict = torch.argmax(out)
    
    
    self._pred = out_predict
    self._label = input_flatten[0]
tp.assign_process_process = assign_process


In [8]:
Hidden,Out = tp.train()

TypeError: rand() received an invalid combination of arguments - got (int, int, require_grads=bool), but expected one of:
 * (tuple of ints size, *, torch.Generator generator, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [104]:
mnist_test.train_data.float().reshape(mnist_test.train_data.shape[0],-1) @ Hidden @ Out

tensor([[1500809.0000],
        [2357211.5000],
        [ 794036.5000],
        ...,
        [2337890.7500],
        [2206151.0000],
        [3377359.0000]], grad_fn=<MmBackward0>)

In [105]:
Out

tensor([[16.3707],
        [15.9951],
        [15.8699],
        [16.2360],
        [16.3383],
        [16.0468],
        [15.9639],
        [16.0041],
        [16.1168],
        [16.3451]], requires_grad=True)